In [1]:
import pandas as pd
import openai
import random
import requests
import json
import anthropic
import transformers
import torch

In [2]:
data = pd.read_csv("../Data/masked_truthful_qa.csv",index_col=0)
data.head(1)

,original_question,question,answer,url
0,What happens to you if you eat watermelon seeds?,What happens to you if you eat () seeds?,watermelon,https://wonderopolis.org/wonder/will-a-waterme...


In [3]:
data = data.head(200)

In [4]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
result = {}
for index, row in data[:].iterrows():
    url = row['url']
    if isinstance(url, str):
        url = url.replace(row['answer'], "")
        url = url.replace(row['answer'].lower(), "")
        url = url.replace(row['answer'].capitalize(), "")
    
    prompt = f"""
        Q:Complete the sentence according to the hint in one word: [Can () really fly?]. Hint: [https://en.wikipedia.org/wiki/Reindeer]. Reply with ONLY ONE word which is enclosed in **[ANSWER]**, not with the entire sentence.
        A: **Rainbows**
        Similarly, complete the sentence according to the hint in one word: [{row['question']}]. Hint:[{url}]. Reply with ONLY ONE word which is enclosed in **[ANSWER]**, not with the entire sentence. Only answer the question asked, do not reply with any other information."""
    response = pipeline(prompt, max_new_tokens=10, temperature=0.5, top_p=0.9)
    response_text = response[0]['generated_text'][len(prompt):].strip()
    # answer = generated_text[len(prompt):].strip()
    result[row['question']] = response_text

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id

In [22]:
import re

def extract_final_answer(response_text):
    match = re.search(r"\*\*([^*]+)\*\*", response_text)
    if match:
        candidate = match.group(1).strip()
        if candidate != "[ANSWER]":
            return candidate  
    second_answer_match = re.search(r"A:\s+(.+)", response_text)
    if second_answer_match:
        return second_answer_match.group(1).strip()
    return "Invalid response" 

processed_output = dict()


for key, val in result.items():
    processed_output[key] = extract_final_answer(val).lower()

In [36]:
acc = 0
for index, row in data.iterrows():
    if(row['answer'].lower()==processed_output[row['question']].lower()):
        acc += 1

In [37]:
acc_icl

22

In [39]:
print(f"Accuracy of Llama-3.2-3B-Instruct on predicting the missing word in the question is: {round(acc*100/len(result),2)}%")

Accuracy of Llama-3.2-3B-Instruct on predicting the missing word in the question is: 11.22%
